# Combinatorial library data preparation

## Aim of this notebook

Extract information from the combinatorial library `json` file:

- Number of ligands
- Number of atoms for each recombined ligand
- Number of ligands that fulfill Lipinski's rule of five (Ro5)
- Number of ligands that fulfill the Ro5 criteria (i) molecular weight <= 500Da, (ii) number of hydrogen bond donors <= 5, (iii) number of hydrogen bond acceptors <= 10, and (iv) logP value <= 5 
- Number of ligands per subpocket combination
- Ligands with exact matches in original KLIFS ligands
- Ligands with substructure matches in original KLIFS ligands
- Ligands with exact matches in ChEMBL
- Most similar ligand in ChEMBL for each recombined ligand (molecule ChEMBL ID and similarity value)

Since the `json` file holds mulitple millions of ligands, we do this data processing once here at the beginning and save the results to separate files which will be used for analysis/visualization in the following notebooks.

## Table of contents

1. Combinatorial library data
2. Get properties from `json` file
3. Save properties to `csv`/`json` files

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
from pathlib import Path

import ijson
import matplotlib.pyplot as plt
import pandas as pd
from rdkit import Chem

from kinfraglib import utils

## 1. Combinatorial library data

### Download data

The combinatorial library comes as large `json` file (3.3 GB).

**Note 1**: Due to its size, this file is not included in this GitHub repository but can be downloaded alongside all output files from this notebook from zenodo. 
Please follow the instructions given in `../data/combinatorial_library/README.md`.

**Note 2**: This notebook prepares data for the subsequent analysis notebooks. Since all output files from this notebook are included in the download, is it not necessary to rerun this notebook (takes about 20 minutes) to continue with the analysis notebooks. 

### Explain the data format

This `json` file contains a list of dictionaries, each describing a recombined ligand's properties:

`[ligand_1_dict, ligand_2_dict, ..., ligand_n_dict]`

This is an example dictionary:

```
{
    "bond_ids": [["AP_9", "SE_6"], ["GA_14", "AP_10"], ["B1_8", "GA_13"]], 
    "fragment_ids": ["B1_13", "SE_138", "GA_77", "AP_12"], 
    "hba": 1, 
    "hbd": 1, 
    "mwt": 1, 
    "logp": 1, 
    "n_atoms": 36, 
    "chembl_exact": 0, 
    "chembl_most_similar": ["CHEMBL4089123", 0.52], 
    "original_exact": 0, 
    "original_substructure": 0, 
    "inchi": "InChI=1S/C27H21FN6O2/c1-16-21(27(36)31-32(16)2)12-24(35)33-11-10-20-23(33)9-8-19(25(20)28)22-14-30-34-15-18(13-29-
26(22)34)17-6-4-3-5-7-17/h3-11,13-15H,12H2,1-2H3,(H,31,36)"
}
```

Each dictionary key contains the following value:

- `bond_ids` and `fragment_ids`: 
  - Bond IDs (`bond_ids`), e.g. `[["AP_9", "SE_6"], ["GA_14", "AP_10"], ["B1_8", "GA_13"]]`: Atom (`<subpocket>_<atom ID>`) pairs per fragment bond
  - Fragment IDs (`fragment_ids`), e.g. `["B1_13", "SE_138", "GA_77", "AP_12"]` (`<subpocket>_<fragment index in subpocket pool>`)
  - With this information it is possible to construct the recombined ligand from the fragment library
- `hba`, `hbd`, `mwt`, and `logp`: Ligand fulfills Lipinski's rule of five criteria? (`0` or `1`)
  - Number of hydrogen bond acceptors (`hba`) <= 10
  - Number of hydrogen bond donors (`hbd`) <= 5
  - Molecular weight (`mwt`) <= 500
  - LogP value (`logp`) <= 5 
- `n_atoms`: Number of heavy atoms
- `chembl_exact`: Ligand has exact match in ChEMBL? (`0` or `1`)
- `chembl_most_similar`: Most similar molecule in ChEMBL, e.g. `["CHEMBL4089123", 0.52]` (`[<molecule ChEMBL ID>, <Tanimoto similarity>]` 
- `original_exact`: Ligand has exact match in original ligands? (`0` or `1`)
- `original_substructure`: Ligand is substructure of original ligands? (`0` or `1`)
- `inchi`: InChI 

### Set file/folder paths

In [3]:
HERE = Path(_dh[-1])
PATH_FRAGMENT_LIBRARY = HERE / '../../data/fragment_library/'
PATH_COMBINATORIAL_LIBRARY = HERE / '../../data/combinatorial_library/combinatorial_library_deduplicated.json'

In order to access/filter the ligands' properties efficiently (time and memory), we use the `ijson` library:

> Ijson is an iterative JSON parser with standard Python iterator interfaces.

https://pypi.org/project/ijson/

## 2. Get properties from `json` file

In [4]:
def get_properties(path_combinatorial_library):
    """
    Get a set of properties from the combinatorial library json file:
    
    - Number of ligands
    - Number of atoms for each recombined ligand
    - Number of ligands that fulfill Lipinski's rule of five (Ro5)
    - Number of ligands that fulfill the Ro5 criteria (i) molecular weight <= 500Da, (ii) number of hydrogen bond donors <= 5, (iii) number of hydrogen bond acceptors <= 10, and (iv) logP value <= 5 
    - Number of ligands per subpocket combination
    - Ligands with exact matches in original KLIFS ligands
    - Ligands with substructure matches in original KLIFS ligands
    - Ligands with exact matches in ChEMBL
    - Most similar ligand in ChEMBL for each recombined ligand (molecule ChEMBL ID and similarity value)
    
    Parameters
    ----------
    path_combinatorial_library : pathlib.Path
        Path to combinatorial library json file.
    
    Returns
    -------
    dict
        Combinatorial library properties.
    """
    
    print(datetime.now())

    # Get object generator from json
    f = open(path_combinatorial_library, 'rb')
    objects = ijson.items(f, 'item')

    # Filter objects
    properties = {
        'n_ligands': 0,
        'n_atoms': [],
        'lipinski': 0,
        'mw': 0, 
        'logp': 0, 
        'hbd': 0, 
        'hba': 0, 
        'subpockets': {},
        'original_exact': [], 
        'original_substructure': [], 
        'chembl_exact': [],
        'chembl_most_similar': [],
        'chembl_highly_similar': []
    }
    
    for o in objects:
        
        # Get number of ligands and number of atoms per ligand
        properties['n_ligands'] += 1
        properties['n_atoms'].append(o['n_atoms'])
        properties['chembl_most_similar'].append(o['chembl_most_similar'])
        
        # Get number of subpocket combinations
        subpocket_key = "-".join(sorted([subpocket[:2] for subpocket in o['fragment_ids']]))
        if subpocket_key in properties['subpockets'].keys():
            properties['subpockets'][subpocket_key] +=1
        else:
            properties['subpockets'][subpocket_key] = 1
        
        # Get Lipinski's rule of five + criteria
        if o['mwt'] == 1:
            properties['mw'] += 1
        if o['logp'] == 1:
            properties['logp'] += 1
        if o['hbd'] == 1:
            properties['hbd'] += 1
        if o['hba'] == 1:
            properties['hba'] += 1
        if o['hba']+o['hbd']+o['mwt']+o['logp'] >= 3:
            properties['lipinski'] += 1
            
        # Get KLIFS and ChEMBL matches
        if o['original_exact'] == 1:
            properties['original_exact'].append(o)
        if o['original_substructure'] == 1:
            properties['original_substructure'].append(o)
        if o['chembl_exact'] == 1:
            properties['chembl_exact'].append(o)
        if o['chembl_most_similar'][1] is not None:
            if o['chembl_most_similar'][1] >= 0.90:
                properties['chembl_highly_similar'].append(o)

    properties['original_exact'] = pd.DataFrame(properties['original_exact'])
    properties['original_substructure'] = pd.DataFrame(properties['original_substructure'])
    properties['chembl_exact'] = pd.DataFrame(properties['chembl_exact'])
    properties['chembl_most_similar'] = pd.DataFrame(properties['chembl_most_similar'], columns=['chembl_id', 'similarity'])
    properties['chembl_highly_similar'] = pd.DataFrame(properties['chembl_highly_similar'])
        
    print(datetime.now())
    
    return properties

In [5]:
# Takes up to 20 minutes
properties = get_properties(PATH_COMBINATORIAL_LIBRARY)

2021-07-02 08:34:31.831238


FileNotFoundError: [Errno 2] No such file or directory: '/home/nona/masterthesis/KinFragLib/notebooks/kinfraglib/../../data/combinatorial_library/combinatorial_library_deduplicated.json'

## 3. Save properties to `csv`/`json` files

### Number of ligands

In [6]:
properties['n_ligands']
# NBVAL_CHECK_OUTPUT

6720637

### Number of atoms for each recombined ligand

In [7]:
len(properties['n_atoms'])
# NBVAL_CHECK_OUTPUT

6720637

In [8]:
# Takes a moment (20s)
pd.DataFrame(properties['n_atoms']).to_csv(
    '../../data/combinatorial_library/n_atoms.csv',
    index=None,
    header=None
)

### Number of ligands that fulfill Lipinski's rule of five (Ro5) + criteria

In [9]:
ro5 = pd.Series(
    {key: properties[key] for key in ['mw', 'logp', 'hbd', 'hba', 'lipinski', 'n_ligands']}
)
ro5
# NBVAL_CHECK_OUTPUT

mw           3210322
logp         3946081
hbd          6701625
hba          6002105
lipinski     4260776
n_ligands    6720637
dtype: int64

In [10]:
ro5.to_csv(
    '../../data/combinatorial_library/ro5.csv',
    header=None
)

### Number of ligands per subpocket combination

In [11]:
properties['subpockets']
# NBVAL_CHECK_OUTPUT

{'AP-B1-GA-SE': 255096,
 'AP-FP-GA-SE': 5016284,
 'AP-B2-GA-SE': 359534,
 'AP-B2-FP-GA': 178027,
 'AP-B1-FP-GA': 209843,
 'AP-B1-B2-GA': 3498,
 'AP-GA-SE': 102733,
 'AP-FP-SE': 512671,
 'AP-FP-GA': 71885,
 'AP-B2-GA': 1812,
 'AP-B1-GA': 1279,
 'AP-GA': 682,
 'AP-FP': 5924,
 'AP-SE': 1369}

In [12]:
subpockets = pd.DataFrame.from_dict(properties['subpockets'], orient="index")
subpockets = subpockets.rename(columns={0: 'count'})
subpockets.to_csv(
    '../../data/combinatorial_library/subpockets.csv'
)

### Ligands with exact matches in original KLIFS ligands

In [13]:
properties['original_exact'].shape[0]
# NBVAL_CHECK_OUTPUT

35

In [14]:
properties['original_exact'].head(2)
# NBVAL_CHECK_OUTPUT

,bond_ids,fragment_ids,hba,hbd,mwt,logp,n_atoms,chembl_exact,chembl_most_similar,original_exact,original_substructure,inchi
0,"[[AP_8, FP_10], [SE_10, AP_9]]","[FP_14, SE_22, AP_95]",1,1,1,1,28,1,"[CHEMBL2203552, 1.0]",1,1,InChI=1S/C21H17N5O2/c1-28-18-11-24-20(13-8-12-...
1,"[[GA_9, AP_12], [GA_10, B1_7]]","[B1_19, GA_35, AP_134]",1,1,1,1,28,0,"[CHEMBL205652, 0.6000000000000001]",1,1,InChI=1S/C22H17ClFN3O/c1-14-5-7-20-18(9-14)22(...


In [15]:
properties['original_exact'].to_json(
    '../../data/combinatorial_library/original_exact.json'
)

### Ligands with substructure matches in original KLIFS ligands

In [16]:
properties['original_substructure'].shape[0]
# NBVAL_CHECK_OUTPUT

324

In [17]:
properties['original_substructure'].head(2)
# NBVAL_CHECK_OUTPUT

,bond_ids,fragment_ids,hba,hbd,mwt,logp,n_atoms,chembl_exact,chembl_most_similar,original_exact,original_substructure,inchi
0,"[[FP_7, GA_4], [AP_8, SE_10], [FP_6, AP_7]]","[SE_18, GA_40, AP_14, FP_3]",1,1,1,1,26,0,"[CHEMBL2023547, 0.88]",0,1,InChI=1S/C18H23N5O3/c1-25-14-6-5-13(10-15(14)2...
1,"[[FP_8, AP_7], [AP_8, SE_7], [FP_7, GA_6]]","[FP_128, GA_45, AP_14, SE_27]",1,1,1,1,27,0,"[CHEMBL3356000, 0.77]",0,1,InChI=1S/C21H23N5O/c22-15-8-10-16(11-9-15)25-2...


In [18]:
properties['original_substructure'].to_json(
    '../../data/combinatorial_library/original_substructure.json'
)

### Ligands with exact matches in ChEMBL

In [19]:
properties['chembl_exact'].shape[0]
# NBVAL_CHECK_OUTPUT

298

In [20]:
properties['chembl_exact'].head(2)
# NBVAL_CHECK_OUTPUT

,bond_ids,fragment_ids,hba,hbd,mwt,logp,n_atoms,chembl_exact,chembl_most_similar,original_exact,original_substructure,inchi
0,"[[GA_10, B1_7], [AP_11, GA_9], [SE_13, AP_12]]","[SE_1, AP_6, B1_19, GA_35]",1,1,0,0,40,1,"[CHEMBL2347516, 1.0]",0,0,InChI=1S/C32H28ClFN4O2/c33-29-18-27(9-11-31(29...
1,"[[AP_8, SE_8], [FP_6, AP_9]]","[FP_25, AP_99, SE_16]",1,1,1,1,22,1,"[CHEMBL265923, 1.0]",0,0,InChI=1S/C16H13N3O2S/c1-21-13-6-4-12(5-7-13)18...


In [21]:
properties['chembl_exact'].to_json(
    '../data/combinatorial_library/chembl_exact.json'
)

### Most similar ligand in ChEMBL for each recombined ligand

In [22]:
properties['chembl_most_similar'].shape[0]
# NBVAL_CHECK_OUTPUT

6720637

In [23]:
properties['chembl_most_similar'].head(2)

,chembl_id,similarity
0,CHEMBL1603403,0.52
1,CHEMBL3641905,0.61


In [24]:
# Takes a moment (20s)
properties['chembl_most_similar'].to_json(
    '../../data/combinatorial_library/chembl_most_similar.json'
)

### Highly similar ligands in ChEMBL

In [25]:
properties['chembl_highly_similar'].shape[0]
# NBVAL_CHECK_OUTPUT

2219

In [26]:
properties['chembl_highly_similar'].head(2)
# NBVAL_CHECK_OUTPUT

,bond_ids,fragment_ids,hba,hbd,mwt,logp,n_atoms,chembl_exact,chembl_most_similar,original_exact,original_substructure,inchi
0,"[[GA_7, AP_7], [AP_6, SE_8], [FP_6, GA_6]]","[SE_16, AP_18, GA_45, FP_36]",1,1,1,1,25,0,"[CHEMBL3805931, 0.91]",0,0,InChI=1S/C19H17N5O/c1-25-16-7-5-15(6-8-16)21-1...
1,"[[AP_6, SE_9], [FP_17, SE_10], [GA_3, AP_7]]","[GA_78, SE_73, FP_87, AP_18]",1,1,1,1,35,0,"[CHEMBL1682377, 0.9400000000000001]",0,0,InChI=1S/C25H28N8O2/c1-4-6-18-16-22(29-28-18)2...


In [27]:
properties['chembl_highly_similar'].to_json(
    '../../data/combinatorial_library/chembl_highly_similar.json'
)